### This nootebook present training model from **MMDET** library on VizDrone2019DET Dataset


In [ ]:
%pip install -U openmim
!mim install mmcv-full

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pip install mmdet

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -v -e .

In [ ]:
import torch, torchvision
from mmdet.apis import init_detector, inference_detector
import mmcv
from mmcv.runner import load_checkpoint
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import draw_bounding_boxes
from torchvision import transforms
from PIL import Image
import sys
import os
from tqdm import trange
import json
from torchvision.ops import box_iou

#Register new dataset and define *load_annotation()* function

#### VizDrone2019Det classes:

* 0:   ignored regions
* 1:   pedestrian
* 2:   people
* 3:   bicycle
* 4:   car
* 5:   van
* 6:   truck
* 7:   tricycle
* 8:   awning-tricycle
* 9:   bus
* 10:  motor
* 11:  others


In [6]:
def get_bbox(path):
    with open(path, 'r') as file:
        annotation = file.read().strip().split('\n')
        annotation_list=list(map(lambda x: list(map(int, x.split(',')[:6])), annotation))
        annotation_list = list(map(lambda a: [a[0], a[1], a[0]+a[2], a[1]+a[3]], list(filter(lambda a: a[5]==2, annotation_list))))
        return annotation_list

In [7]:
path_val='/content/drive/MyDrive/VisDrone2019-DET-val/'
path_train='/content/drive/MyDrive/VisDrone2019-DET-train/'
path_test='/content/drive/MyDrive/VIzDrone2019DET-test/'

MMDetection middle format of data

```python
[
    {
        'filename': 'a.jpg',
        'width': 1280,
        'height': 720,
        'ann': {
            'bboxes': <np.ndarray> (n, 4),
            'labels': <np.ndarray> (n, ),
            'bboxes_ignore': <np.ndarray> (k, 4), (optional field)
            'labels_ignore': <np.ndarray> (k, 4) (optional field)
        }
    },
    ...
]
```

In [ ]:
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
import numpy as np
from tqdm import tqdm

@DATASETS.register_module()
class VizdroneDataset(CustomDataset):
      CLASSES = ('Person', )

      def load_annotations(self, ann_file):
          cat2label = {k: i for i, k in enumerate(self.CLASSES)}
          image_list = os.listdir(f'{self.img_prefix}')
          data_infos = []
          for image_name in tqdm(image_list):
              img_path = f'{self.data_root}images/{image_name}'
              image = mmcv.imread(img_path)
              height, width = image.shape[:2]
    
              data_info = dict(filename=image_name, width=width, height=height)
              # load annotations
              label_prefix = f'{self.data_root}/annotations/'
              file_name = image_name.split('.')[0] + '.txt'
              
              bounding_boxes = np.array(get_bbox(f'{label_prefix}{file_name}'), 
                                dtype=np.float32).reshape(-1, 4)
              data_anno = dict(
                  bboxes=bounding_boxes,
                  labels=np.array([cat2label['Person'] 
                                   for _ in range(bounding_boxes.shape[0])], dtype=np.long),
              )
              data_info.update(ann=data_anno)
              data_infos.append(data_info)
          return data_infos

# Load and modify config file

In [8]:
from mmcv import Config
cfg = Config.fromfile('./configs/dcn/cascade_rcnn_r101_fpn_dconv_c3-c5_1x_coco.py')

In [ ]:
from mmdet.apis import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VizdroneDataset'
cfg.data_root = '/content/drive/MyDrive/'
#Test data configurations
cfg.data.test.type = 'VizdroneDataset'
cfg.data.test.data_root = '/content/drive/MyDrive/VIzDrone2019DET-test/'
cfg.data.test.img_prefix = 'images/'
#Train data configurations
cfg.data.train.type = 'VizdroneDataset'
cfg.data.train.data_root = '/content/drive/MyDrive/VisDrone2019-DET-train/'
#tranformations
cfg.data.train.pipeline = [
     {'type': 'LoadImageFromFile'},
     {'type': 'LoadAnnotations', 'with_bbox': True},
     {'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True},
     {'type': 'RandomFlip', 'flip_ratio': 0.5, 'direction': ['horizontal']},
     {'type': "Albu", 'transforms': [
         dict(type='RandomBrightnessContrast', brightness_limit=0.2,
              contrast_limit=0.2, p=0.5),
         dict(type='GaussNoise'),
         dict(type='ShiftScaleRotate', shift_limit=0.0625,
              scale_limit=0.1, rotate_limit=45, p=0.4),
     ],
      'bbox_params': 
         dict(
             type='BboxParams',
             format='pascal_voc',
             
             label_fields=['gt_labels'],
             min_visibility=0.0,
             filter_lost_elements=True),
         'keymap': {
         'img': 'image',
         'gt_bboxes': 'bboxes'
         }
    },
    {'type': 'CutOut', 'n_holes': (0, 3), 'cutout_ratio': (0.05, 0.2)},
    {'type': 'Normalize',
     'mean': [123.675, 116.28, 103.53],
     'std': [58.395, 57.12, 57.375],
     'to_rgb': True},
    {'type': 'Pad', 'size_divisor': 32},
    {'type': 'DefaultFormatBundle'},
    {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}
]
cfg.data.train.img_prefix = 'images/'

cfg.data.val.type = 'VizdroneDataset'
cfg.data.val.data_root = '/content/drive/MyDrive/VisDrone2019-DET-val/'
cfg.data.val.img_prefix = 'images/'
# modify num classes of the model in box head
for layer in cfg.model.roi_head.bbox_head:
    layer.num_classes = 1
#Path to load pretrained model
cfg.load_from = '/content/drive/MyDrive/training_on_vizdrone/model2.pth'
# Set up working dir to save files and logs.
cfg.work_dir = '/content/drive/MyDrive/training_on_vizdrone/'

# The original learning rate (LR) is set for 8-GPU training.
cfg.optimizer.lr = 0.02 / 12
cfg.lr_config.warmup_ratio = 1e-4
cfg.log_config.interval = 10

cfg.custom_imports = dict(
     imports=['mmdet.datasets.VizdroneDataset'],
     allow_failed_imports=False)
# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1
# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = 'cuda'
cfg.runner.max_epochs = 6
# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


# Build detector, dataset and start training

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES

In [ ]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

# Save configs and test detector on test data

ATTENTION!!! \
To test model its necessary create file with *class VizdroneDataset(CustomDataset)* defenition and put it into mmdetection/mmdet/datasets/VizdroneDataset.py

In [18]:
cfg.dump('/content/configs.py')

In [19]:
cfg = Config.fromfile('/content/configs.py')

In [ ]:
!python tools/test.py \
    /content/configs.py \
    /content/drive/MyDrive/training_on_vizdrone/model2.pth \
    --eval mAP --work-dir /content/metrics